# Project: **Newton, Bisect, and Secant Method** in **Options-Modelling**

## About the Software

The software I found was a stock option modelling software that took advantage of the Black-Scholes model for pricing option contracts. For those who do not know, an option contract is known as a derivative of the underlying stock price, and they can be used to hedge against positions or to increase returns on positions. For the Black-Shcoles model (which is just one of several option pricing models), the price of the contract can be calculated from the following function:

$$C = S_{t}N(d_{1}) - Ke^{-rt}N(d_{2})$$ 

Where,

$$d_1 = \frac{ln(\frac{S_{t}}{K}) + (r + \frac{\sigma^{2}_{v}}{2})t }{\sigma_{s} \sqrt{t}}$$

and

$$d_2 = d_1 - \sigma_{s}t$$

where,

$$C = \text{Call Option Price}$$
$$S = \text{Current Stock Price}$$
$$K = \text{Strike Price}$$
$$r = \text{risk free interest rate}$$
$$t = \text{time to maturity}$$
$$N = \text{A normal distribution}$$
$$v = \text{volatility}$$

Below is how the author implements the Black-Scholes model:

In [160]:
def norminv(x):
    return ((1.0/math.sqrt(2.0*math.pi)) * math.exp(-x*x*0.5))

#calculate d1
def d1(S0, K, r, T, sigma, q):
    deno = (sigma * math.sqrt(T))
    if (deno==0):
        return 0
    logReturns = math.log(S0/float(K)) if ((S0/float(K)) > 0.0) else 0.0
    return (float(logReturns) + (float(r) - float(q) + float(sigma)*float(sigma)*0.5)*float(T)) / float(deno)
    
#calculate d2
def d2(S0, K, r, T, sigma, q):
        return d1(S0, K, r, T, sigma, q)-sigma*math.sqrt(T)

#Black-Scholes
def bsformula(callput, S0, K, r, T, sigma, q=0):
    N = stats.norm.cdf
                
    def optionValueOfCall(S0, K, r, T, sigma, q):       
        _d1 = d1(S0, K, r, T, sigma, q)
        _d2 = d2(S0, K, r, T, sigma, q)
        return S0*math.exp(-q*T)*N(_d1)- K*math.exp(-r*T)*N(_d2)
      
    def optionValueOfPut(S0, K, r, T, sigma, q):
        _d1 = d1(S0, K, r, T, sigma, q)
        _d2 = d2(S0, K, r, T, sigma, q)
        return float(K)*math.exp(-float(r)*float(T))*N(-_d2) - float(S0)*math.exp(-float(q)*float(T))*N(-_d1)
        
    def delta(callput, S0, K, r, T, sigma, q):
        _d1 = d1(S0, K, r, T, sigma, q)        
        if callput.lower() == "call":            
            return N(_d1) * math.exp(-q*T)
        else:
            return (N(_d1)-1)* math.exp(-q*T)
    
    def vega(S0, K, r, T, sigma, q):
        _d1 = d1(S0, K, r, T, sigma, q)
        return S0  * math.sqrt(T) * norminv(_d1)  * math.exp(-q*T)
    
    if callput.lower()=="call":
        optionValue = optionValueOfCall(S0, K, r, T, sigma, q)
    else:
        optionValue = optionValueOfPut(S0, K, r, T, sigma, q)
        
    _delta = delta(callput, S0, K, r, T, sigma, q)
    _vega = vega(S0, K, r, T, sigma, q)
    
    return (optionValue, _delta, _vega)

However, one of the most important factors in an options price is called Implied Volatility (IV). Many traders are more interested in knowing the IV of a certain option contract rather than the price itself because they know the latter is determined by the former, and determining what volatility level the option was priced at can uncover unique trading strategies. However, IV is the only input to the model that is not directly observable, so the model must be sovled for implied volatility.

Now, if implied volatility is used to determine the price of an option, then if we know the price of the contract we can use that to find the implied volatility level. This creates the parameters and opportunity to use a root finding algorithm (Bisection, Newton-Raphson, Secant) to find the right implied volatility level where:
   
$$f(IV) = C$$

Where,

$$f = \text{Black-Scholes Model}$$
$$IV = \text{Implied Volatility}$$
$$C = \text{Current price of the option}$$

The software repository I found uses the root finding algorithms mentioned above to find the current level of implied volatility to accurately price an option. 


The software is written in python and can be found here:

https://github.com/jamesmawm/Options-Modelling

This is not  a popular or extremley active repository by any means, but I found it intersting seeing these rootfinding methods being applied in options modeling.

Another instance of roofinding being used in finding IV can be found here:

https://github.com/mcdallas/wallstreet/blob/master/wallstreet/blackandscholes.py

However, in this repository they use the scipy method:

In [3]:
  def implied_volatility(self):
        impvol = lambda x: self.BS(self.S, self.K, self.T, x, self.r, self.q) - self.opt_price
        iv = fsolve(impvol, SOLVER_STARTING_VALUE, fprime=self._fprime, xtol=IMPLIED_VOLATILITY_TOLERANCE)
        return iv[0]

instead of coding it all out.

## About the method

The methods used are all algorithms we learned about near the beginning of the semester. In simple terms these methods are attemping to find 

$$x$$

of a function $f$ where,

$$f(x) = 0$$

In our current situation of finding IV we want to solve

$$f(x) = |C(\sigma_{i}) - C(\sigma_{i+1})| = 0$$

where,

$$C = \text{price of option at given IV}$$
$$\sigma_{i} = \text{current IV guess}$$
$$\sigma_{i+1} = \text{IV guess after next iteration}$$

This method is used in the current context because if traders can find accurate estimates for the implied volatity, then it can lead to more profitable and consistent trading. We must use rootfinding algorithms because, as stated above, the implied volatility is not directly observalbe from the model itself. These rootfinding algorithms allows for efficient and fast startegies to finding an accurate estimate for IV. The only other option would be to iteratively increase the IV guess from $0\%$ until you reach the desired IV level. However, this is extremely inefficient compared to the rootfinding strategies.

### Questions

Can you still use Newton's method to find $x$ such that $f(x) = y$ for a specific $y$? Or can you only use it to find $f(x) = 0$?

## Method as it appears in the software

The role of the method in this software, as stated above, is to find the implied volatility of a given option contract.



In [157]:
def newtonsMethod(callput, S0, K, r, T, q, price, initialVol, tolerance=0.01, maxIterations=100):
    iterations=0
    x = initialVol
    prevega=0
    while iterations < maxIterations:
        bsdata = bsformula(callput, S0, K, r, T, x, q)
        optionValue = bsdata[0]
        vega = bsdata[2]
        
        #Take the current implied volatility to prevent division by zero error.
        if (vega==0):   
            return (x, iterations+1)
        prevega = vega
        newt = float(x) - (float(optionValue)-float(price))/float(vega)
        if (abs(newt-x)<tolerance):            
            return (newt, iterations+1)
        
        x = newt        
        iterations += 1
        
    return (float('NaN'), 0)

def secantMethod(targetfunction, x0,x1,n=100):
    numberOfCalls = 0
    for i in range(n):        
        numberOfCalls+=1
        ans1 = targetfunction(x1)
        ans2 = targetfunction(x0)            
        deno = ans1-ans2
        
        if deno == 0.0:           
            return (x1, numberOfCalls)
        x_temp = x1- (ans1*(x1-x0))/deno
        x0 = x1
        x1 = x_temp        
        
    return (x1, numberOfCalls)

Above is how the author implements Newton's method and secant method. As we can see this is pretty much classical Newton. However, the only thing that might be confusing is vega. Vega is a value associated with an option contract that tells how much the price of an option contract will change with a $1\%$ change in implied volatility. So, essentially it is the the derivative of implied volotility with respect to option price. Because of this we can use vega as the derivative of IV for Newton's method. We can see that every iteration they call the Black-Scholes method to find the option value corresponding to the estimated implied volatility and then updates the implied volotility if the calculated option price is not within a given tolerance.

Also, the implemented secant method is just the classical algorithm with the target function being the Black-Scholes model and $x0$ and $x1$ are estimated IV values. 

Below is how one can use these functions. Here we are first creating the theoretical option price with an underlying stock price of $\$100$, a strike price of $\$110$, risk free rate of 5%, 30 days untill expiration, and a given implied volatility of 3%.

We then take the outputted option price from the Black-Scholes algorithm and then input that price along with the other given parameters to see if the $bsimpvol$ function can accurately find the given implied volatility using all three rootfinding algorithms.

In [156]:
from BS import *

callput = "call"
S0 = 100
K = 110
r = 0.05
T = 30
sigma = 0.3 #given implied volatility

price = bsformula(callput, S0, K, r, T, sigma)

print("Option price: ${}".format(price[0]), '\n')

#find implied volatility with initial guess 0.5
algos = ['newton', 'bisect', 'secant']
for algo in algos:
    imp_vol, calls = bsimpvol(callput, S0, K, r, T, price[0], sigma=0.5, q=0, priceTolerance=0.01, method=algo, reportCalls=True)
    print("Method: {}".format(algo))
    print("Implied Volatility: {}%".format(imp_vol*100))
    print("Iterations: {}".format(calls))
    print('\n')

Option price: $82.71950447564355 

Method: newton
Implied Volatility: 30.000045099661378%
Iterations: 3


Method: bisect
Implied Volatility: 29.999999999999993%
Iterations: 19


Method: secant
Implied Volatility: 30.000000000000014%
Iterations: 5




As we can see Newton's method converged the fastest in only 3 iterations, however the secant method had the most accurate final estimate of the implied volatility. Bisect was less accurate and took longer in comparison to both Newton's and secant methods. 

You can also change any of the starting parameters and initial guess to see how the option price changes and how the efficiency and accuracy of the methods change (the sigma parameter in the bsimpvol function call only effects the starting guess for Newton's method).